<a href="https://colab.research.google.com/github/pablobermudez/SOA_EA3/blob/master/Ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introducción

El ejercicio posee dos versiones del algoritmo de ordenamiento bubble sort, la primera de ellas deberá ejecutarse en el entorno CPU con el lenguaje python y la segunda en el entorno GPU utilizando el lenguaje cuda junto con la libreria pycuda.

La finalidad del ejercicio no es centrarse en la complejidad del algoritmo utilizado, sino en la información que nos provee y en el sustento teórico que poseen.

A priori, ya investigado el tema, sabemos que algoritmos como bubble sort no son algoritmos idoneos para ser ejecutados por el GPU. Esto es debido a que los hilos del kernel deberán sincronizarse obligatoriamente para poder realizar su tarea de manera coordinada y efectiva.

Para entender mejor lo mencionado, ejecutaremos la prueba y veremos entonces las conclusiones.




#2 Armado del ambiente
Instalar en el cuaderno el módulo CUDA de Python.


In [ ]:
!pip install pycuda

#3.1 Desarrollo - CPU



In [ ]:
# --------------------------------------------
#@title 3.1.1 Parámetros de ejecución { vertical-output: true }

cantidad_N =   6000#@param {type: "number"}
# --------------------------------------------

# --------------------------------------------
# Valido número de ingreso por el usuario
if (cantidad_N == 0):
  sys.exit("La cantidad de números en el vector debe ser mayor que 0")


from datetime import datetime
tiempo_total = datetime.now()

import random
import numpy as np

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# --------------------------------------------
# Defino variables
arr = [random.randint(1,10) for _ in range(cantidad_N)]

# Defino algoritmo
def bubbleSort(arr): 
    n = len(arr) 
    for i in range(n-1): 
        for j in range(0, n-i-1): 
            if arr[j] > arr[j+1] : 
                arr[j], arr[j+1] = arr[j+1], arr[j] 
  
bubbleSort(arr) 


# TODO - Informo tiempos, hilos y bloques.
tiempo_total = datetime.now() - tiempo_total
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )

# CPU - Informo el resutlado.
print( "------------------------------------")
print( "Array: " )
print( arr )



#3.2 Desarrollo - GPU


In [ ]:
# --------------------------------------------
#@title 3.2.1 Parámetros de ejecución { vertical-output: true }

cantidad_N =   300#@param {type: "number"}
# --------------------------------------------

from datetime import datetime
tiempo_total = datetime.now()

import sys
import random
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

# --------------------------------------------
# Valido número de ingreso por el usuario
if (cantidad_N == 0):
  sys.exit("La cantidad de números en el vector debe ser mayor que 0")


# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# CPU - Defino la memoria de los vectores en cpu.
x_cpu = [random.randint(1,10) for _ in range(cantidad_N)]
x_cpu = np.array(x_cpu, dtype=np.int32)
r_cpu = numpy.empty_like( x_cpu )

# CPU - reservo la memoria GPU.
x_gpu = cuda.mem_alloc( x_cpu.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( x_gpu, x_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""

#include <stdio.h>
__global__ void bubble_sort_gpu( int n, float *X)
{

   int i = threadIdx.x + blockDim.x * blockIdx.x;
   float aux;
    for (int p = 0; p < n; ++p) {
        if ((i - p) % 2 && i < n - 1 && X[i] > X[i + 1]) {
            aux = X[i];
            X[i] = X[i + 1];
            X[i + 1] = aux;
        }     
        __syncthreads();
    }
}

""") 
# CPU - Genero la función kernel.
kernel = module.get_function("bubble_sort_gpu")
tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (cantidad_N+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(cantidad_N), x_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( r_cpu, x_gpu )

tiempo_total = datetime.now() - tiempo_total

# TODO - Informo tiempos, hilos y bloques.
print( "Cantidad de elementos: ", cantidad_N )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )

# CPU - Informo el resutlado.
print( "------------------------------------")
print( "X: " )
print( x_cpu )
print( "------------------------------------")
print( "R: " )
print( r_cpu )

#4 Tabla de Pasos

# 6 Referencias


*   https://dergipark.org.tr/en/download/article-file/225714
* https://docs.nvidia.com/cuda/cuda-math-api/group__CUDA__MATH__SINGLE.html#group__CUDA__MATH__SINGLE
* https://documen.tician.de/pycuda/driver.html
* https://arxiv.org/ftp/arxiv/papers/1505/1505.07605.pdf
